# Seed 생성 확인용 테스트 노트북

dbt seed로 생성된 seed 테이블 확인용

In [ ]:
import sys
sys.path.append('/app')

from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

# Spark 세션 생성
spark = get_spark_session("GDELT_Seed_Test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

In [ ]:
# 1. Spark Metastore에 존재하는 모든 데이터베이스(스키마) 목록을 확인합니다.
print("📖 사용 가능한 데이터베이스(스키마) 목록:")
spark.sql("SHOW DATABASES").show()

In [ ]:
# 2. 'default' 스키마 안에 있는 테이블 목록을 확인합니다.
# dbt seed로 생성된 테이블이 여기에 보여야 합니다. (gdelt_silver_events는 seed 테이블이 아닙니다.)
print("📜 'default' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN default").show()

In [ ]:
# 3. 총 11개의 seed 테이블의 샘플 데이터를 5개씩 출력하여 내용 확인
seed_tables = [
    "actor_ethnic_group_codes",
    "actor_organization_codes",
    "actor_religion_codes",
    "actor_role_codes",
    "event_root_codes",
    "event_detail_codes",
    "event_quad_class_codes",
    "geo_adm_codes",
    "geo_country_fips_codes",
    "geo_country_iso_codes",
    "geo_type_codes"
]

for table in seed_tables:
    print(f"\n 🔍 {table} 테이블 샘플 데이터")
    try:
        spark.sql(f"SELECT * FROM default.{table} LIMIT 5").show(truncate=False)
    except Exception as e:
        print(f"❌ 테이블 조회 중, 오류 발생: {e}")

생성된 seed 테이블을 삭제할 경우, 아래 코드에 "삭제할 seed 테이블명"을 입력하여 실행

In [ ]:
# 삭제할 seed 테이블 이름 목록
seed_tables_to_drop = [
    # "[삭제할 seed 테이블명]"
]
print("🗑️ 기존 Seed 테이블 삭제를 시작합니다.")

for table_name in seed_tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS default.{table_name}")
        print(f"  - 테이블 'default.{table_name}' 삭제 완료.")
    except Exception as e:
        print(f"  - 테이블 'default.{table_name}' 삭제 중 오류 발생: {e}")

print("\n✅ 모든 Seed 테이블 삭제 작업이 완료되었습니다.")

# 삭제 후 테이블 목록을 다시 확인하여 깨끗해졌는지 확인
print("📜 현재 'default' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN default").show()

Staging 폴더의 stg_seed_mapping.sql을 dbt run한 결과 확인

In [ ]:
# stg_seed_mapping 모델 결과 확인
print("📜 [stg_seed_mapping] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM silver.stg_seed_mapping LIMIT 20").show(truncate=False)

In [ ]:
# stg_seed_actors_parsed 모델 결과 확인
print("📜 [stg_seed_actors_parsed] 뷰 : 상위 20개 데이터")
spark.sql("SELECT * FROM silver.stg_seed_actors_parsed LIMIT 20").show(truncate=False)

In [ ]:
# spark.stop()
print("\n✅ Spark 세션이 종료되었습니다.")